# Packed Ensemble Application to the AirfRANS dataset

### Generic Step (Load the required data) <a id='generic_step'></a>

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS) 

In [1]:
# !pip install -r requirements.txt
# or 
# !pip install -U .

Install the AirfRANS package

In [2]:
# !pip install airfrans

### Generic Step (Load the required data) <a id='generic_step'></a>

In [1]:
import os
from lips import get_root_path

In [2]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../ml4physim_startingkit/Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [3]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations","benchmarks","confAirfoil.ini") #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations","simulators","torch_fc.ini") #Configuration file re

Download the data

In [4]:
from lips.dataset.airfransDataSet import download_data
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [5]:
# Load the required benchmark datasets
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path = LOG_PATH)
benchmark.load(path=DIRECTORY_NAME)

Loading dataset (task: reynolds, split: test): 100%|██████████| 496/496 [01:56<00:00,  4.26it/s]


### Training a simple Packed MLP

##### STEP 1: Architecture implementation using the ``torch-uncertainty`` package

In [32]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from torch_uncertainty.layers import PackedLinear

class PackedMLP(nn.Module):
    """
    A simple MLP with packed layers

    Parameters
    ----------
    name: str (default: "PackedMLP")
        The name of the model
    input_size: int (default: None)
        The size of the input
    output_size: int (default: None)
        The size of the output
    hidden_sizes: tuple (default: (100, 100,))
        The sizes of the hidden layers
    activation: torch.nn.functional (default: F.relu)
        The activation function
    dropout: bool (default: False)
        Whether to use dropout
    batch_normalization: bool (default: False)
        Whether to use batch normalization
    M: int (default: 4)
        The number of estimators
    alpha: int (default: 2)
        The alpha parameter
    gamma: int (default: 1)
        The gamma parameter
    device: str (default: "cpu")
        The device to use
    """
    def __init__(self,
                 name: str="PackedMLP",
                 input_size: int=None,
                 output_size: int=None,
                 hidden_sizes: tuple=(100, 100,),
                 activation=F.relu,
                 dropout: bool=False,
                 M: int=4,
                 alpha: int=2,
                 gamma: int=1,
                 device: str="cpu") -> None:
        super().__init__()

        # dropout
        if dropout:
            self.dropout = nn.Dropout(p=0.2)
        else:
            self.dropout = nn.Identity()

        self.name = name
        self.device = device
        
        self.activation = activation
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes

        self.num_estimators = M

        self.input_layer = PackedLinear(self.input_size, self.hidden_sizes[0], alpha=alpha, num_estimators=M, gamma=gamma, first=True,
                                        device=device)
        self.hidden_layers = nn.ModuleList([PackedLinear(in_f, out_f, alpha=alpha, num_estimators=M, gamma=gamma, device=device) \
                                           for in_f, out_f in zip(self.hidden_sizes[:-1], self.hidden_sizes[1:])])
        self.output_layer = PackedLinear(self.hidden_sizes[-1], self.output_size, alpha=alpha, num_estimators=M, gamma=gamma, last=True,
                                         device=device)

    def forward(self, data):
        """
        The forward pass of the model
        """
        out = self.input_layer(data)
        for _, fc_ in enumerate(self.hidden_layers):
            out = fc_(out)
            out = self.activation(out)
            out = self.dropout(out)
        out = self.output_layer(out)
        return out

##### STEP 2: Process the data to acquire the right Inputs and Outputs for the model alongside their dimensions
This function uses a functionality offered by the Dataset class to extract the required inputs and outputs for the problem in hand, which facilitate the task. 

It also allows to create DataLoader from existing datasets.

In [44]:
def process_dataset(dataset, batch_size: int=128000, training: bool=False, shuffle: bool=False, n_workers: int=0):
    if training:
        batch_size = batch_size
        extract_x, extract_y = dataset.extract_data()
    else:
        batch_size = batch_size
        extract_x, extract_y = dataset.extract_data()

    torch_dataset = TensorDataset(torch.from_numpy(extract_x).float(), torch.from_numpy(extract_y).float())
    data_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=n_workers)
    return data_loader

def infer_input_output_size(dataset):
    *dim_inputs, output_size = dataset.get_sizes()
    input_size = np.sum(dim_inputs)
    return input_size, output_size

##### STEP 3: Implementation of the Training, Validation and Prediction functions

**train.** This function allows to train (adjust the parameters of) your defined model using the provided datasets.

**validate.** This function allows to validate your model on a validation dataset. The validation step is not mendatory and is used only to trace the model behavior (overfitting or not). 

**predict.** This function allows to predict using the trained model. The `DataSet` class provides a function `reconstruct_output` which allows to reshape the predictions in the correct form which will be comparable with ground truth. 

In [45]:
from tqdm import tqdm
from einops import rearrange

def train(model, train_loader, val_loader=None, epochs=100, lr=3e-4, device="cpu"):
    train_losses = []
    val_losses = []
    # select your optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # select your loss function
    loss_function = nn.MSELoss()
    pbar = tqdm(range(epochs), desc="Epochs")
    for epoch in pbar:
        # set your model for training
        model.train()
        total_loss = 0
        # iterate over the batches of data
        pbar_batch=tqdm(train_loader)
        for batch in pbar_batch:
            data, target = batch
            # transfer your data on proper device. The model and your data should be on the same device
            data = data.to(device)
            target = target.to(device)
            # reset the gradient
            optimizer.zero_grad()
            # predict using your model on the current batch of data
            prediction = model(data)
            # compute the loss between prediction and real target, by repeating the target so it fits the different estimators 
            loss = loss_function(prediction, target.repeat(model.num_estimators, 1))
            # compute the gradient (backward pass of back propagation algorithm)
            loss.backward()
            # update the parameters of your model
            optimizer.step()
            total_loss += loss.item() * len(data)
        # the validation step is optional
        if val_loader is not None:
            val_loss = validate(model, val_loader, device)
            val_losses.append(val_loss)
        mean_loss = total_loss / len(train_loader.dataset)
        print(f"Train Epoch: {epoch}   Avg_Loss: {mean_loss:.5f}")
        train_losses.append(mean_loss)
    return model, train_losses, val_losses

def validate(model, val_loader, device):
    # set the model for evaluation (no update of the parameters)
    model.eval()
    total_loss = 0
    loss_function = nn.MSELoss()
    with torch.no_grad():
        pbar_batch=tqdm(val_loader)
        for batch in val_loader:
            data, target = batch
            data.to(device)
            target.to(device)
            prediction = model(data)
            loss = loss_function(prediction, target)
            total_loss += loss.item()*len(data)
        mean_loss = total_loss / len(val_loader.dataset)
        print(f"Eval:   Avg_Loss: {mean_loss:.5f}")
    return mean_loss

def predict(model, dataset, device):
    # set the model for the evaluation
    model.eval()
    predictions = []
    observations = []
    test_loader = process_dataset(dataset, training=False, shuffle=False)
    # we dont require the computation of the gradient
    with torch.no_grad():
        for batch in tqdm(test_loader):
            data, target = batch
            data = data.to(device)
            target = target.to(device)
            prediction = model(data)
            
            #averaging the predictions of the different ensemble models
            packed_split = rearrange(prediction, '(n b) m -> b n m', n=model.num_estimators)
            packed_prediction = packed_split.mean(dim=1)

            if device == torch.device("cpu"):
                predictions.append(packed_prediction.numpy())
                observations.append(target.numpy())
            else:
                predictions.append(packed_prediction.cpu().data.numpy())
                observations.append(target.cpu().data.numpy())
    # reconstruct the prediction in the proper required shape of target variables
    predictions = np.concatenate(predictions)
    predictions = dataset.reconstruct_output(predictions)
    # Do the same for the real observations
    observations = np.concatenate(observations)
    observations = dataset.reconstruct_output(observations)

    return predictions, observations

In [46]:
train_loader = process_dataset(benchmark.train_dataset, training=True, n_workers=6)
input_size, output_size = infer_input_output_size(benchmark.train_dataset)

In [47]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = PackedMLP(input_size=input_size,
                               output_size=output_size,
                               hidden_sizes=(50,100,50),
                               activation=F.relu,
                               device=device,
                               dropout=True,
                               )
model.to(device)
model.device

device(type='cuda', index=0)

In [48]:
print(model)

PackedMLP(
  (dropout): Dropout(p=0.2, inplace=False)
  (input_layer): PackedLinear(
    (conv1x1): Conv1d(7, 100, kernel_size=(1,), stride=(1,))
  )
  (hidden_layers): ModuleList(
    (0): PackedLinear(
      (conv1x1): Conv1d(100, 200, kernel_size=(1,), stride=(1,), groups=4)
    )
    (1): PackedLinear(
      (conv1x1): Conv1d(200, 100, kernel_size=(1,), stride=(1,), groups=4)
    )
  )
  (output_layer): PackedLinear(
    (conv1x1): Conv1d(100, 16, kernel_size=(1,), stride=(1,), groups=4)
  )
)


In [49]:
model, train_losses, _ = train(model, train_loader, epochs=1, device=device, lr=3e-4)

Epochs: 100%|██████████| 1/1 [00:30<00:00, 30.21s/it]

Train Epoch: 0   Avg_Loss: 1510000.78315


##### prediction on `test_dataset`
This dataset has the same distribution as the training set

In [69]:
predictions, observations = predict(model, benchmark._test_dataset, device=device)

100%|██████████| 281/281 [04:21<00:00,  1.07it/s]


In [94]:
print("Prediction dimensions: ", predictions["x-velocity"].shape, predictions["y-velocity"].shape, predictions["pressure"].shape, predictions["turbulent_viscosity"].shape)
print("Observation dimensions:", observations["x-velocity"].shape, observations["y-velocity"].shape, observations["pressure"].shape, observations["turbulent_viscosity"].shape)
print("We have good dimensions!")

Prediction dimensions:  (35849332,) (35849332,) (35849332,) (35849332,)
Observation dimensions: (35849332,) (35849332,) (35849332,) (35849332,)
We have good dimensions!


In [72]:
from lips.evaluation.airfrans_evaluation import AirfRANSEvaluation

evaluator = AirfRANSEvaluation(config_path = BENCH_CONFIG_PATH,
                               scenario = BENCHMARK_NAME,
                               data_path = DIRECTORY_NAME,
                               log_path = LOG_PATH)

observation_metadata = benchmark._test_dataset.extra_data
metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 35849332 but corresponding boolean dimension is 18515415

##### Prediction on `test_ood_dataset`
This dataset has a different distribution in comparison to the training set. 

In [ ]:
predictions, observations = predict(model, benchmark._test_ood_dataset, device=device)
evaluator = AirfRANSEvaluation(config_path = BENCH_CONFIG_PATH,
                               scenario = BENCHMARK_NAME,
                               data_path = DIRECTORY_NAME,
                               log_path = LOG_PATH)

metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)

# Model selection (Cross validation)

Create cross validation on hyperparameters of the model

In [60]:
# define the grid of parameters using a logaritmic scale
param_grid = {'lr': [3e-4],
                'hidden_sizes': [(50, 100, 50), (100, 200, 100), (200, 400, 200)],
                'dropout': [True, False]}

hyperparameter_grid = [(hidden_layer_sizes, lr, dropout)
                       for hidden_layer_sizes   in param_grid["hidden_sizes"]
                       for lr                   in param_grid["lr"]
                       for dropout              in param_grid["dropout"]]

for hyperparameter in tqdm(hyperparameter_grid):
    model = PackedMLP(input_size=input_size,
                                output_size=output_size,
                                hidden_sizes=hyperparameter[0],
                                activation=F.relu,
                                device=device,
                                dropout=hyperparameter[2],
                                )
    model.to(device)

    #train using CV
    

In [ ]:
"""Function to implement"""


# Define the function to optimize the hyperparameters
def optimize_hyperparameters(loss_fn, X_train, y_train, X_test, y_test, X_val, y_val, param_dist):
    """
    Searches for the best hyperparameters, using a gridsearch approach
    """
    best_validation_error = np.inf
    best_hyperparameters = None

    input_size  = X_train.shape[1]
    output_size = y_train.shape[1]

    hyperparameter_grid = [(hidden_layer_sizes, activation, lr, batch_size, n_epochs)
                       for hidden_layer_sizes   in param_dist["hidden_layer_sizes"]
                       for activation           in param_dist["activation"]
                       for lr                   in param_dist["lr"]
                       for batch_size           in param_dist["batch_size"]
                       for n_epochs             in param_dist["n_epochs"]]

    n_total = len(hyperparameter_grid)
    counter = 1

    for hidden_layer_sizes, activation, lr, batch_size, n_epochs in hyperparameter_grid:
        print(f"\nstep: {counter}/{n_total}")
        print(f"hidden_layer_sizes: {hidden_layer_sizes}, activation:{activation}, lr{lr}, batch_size:{batch_size}, n_epochs:{n_epochs}")

        model = create_model(input_size, output_size, hidden_layer_sizes, activation)
        train(model, X_train, X_test, y_train, y_test, lr, batch_size, n_epochs, loss_fn)

        # Evaluate the best model on validation data
        y_pred = model(X_val)
        val_mse = loss_fn(y_val, y_pred)

        if (val_mse < best_validation_error):
            best_hyperparameters = [hidden_layer_sizes, activation, lr, batch_size, n_epochs]
            best_weights = copy.deepcopy(model.state_dict())

        training_error  = loss_fn(model(X_train), y_train)
        test_error      = loss_fn(model(X_test), y_test)
        val_error       = loss_fn(model(X_val), y_val)

        print(f"training error: {training_error} , test error: {test_error} , val error: {val_error}")
        counter += 1
    
    best_model = create_model(input_size, output_size, best_hyperparameters[0], best_hyperparameters[1])
    return best_hyperparameters, best_model, best_validation_error